In [2]:
# -*- coding: utf-8 -*-
"""kobertmodel.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1349kWp-wkWywvgzAbJH91djHGExijQ0U
"""

'kobertmodel.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1349kWp-wkWywvgzAbJH91djHGExijQ0U\n'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ox5498mi
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ox5498mi
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [6]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
#GPU 사용

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()


using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/data/total_set.csv')
len(chatbot_data)

chatbot_data.head(5)

,newsfulltext,theme
0,사진김캐디버핏서울은 스장을 중심으로 모객하는 기존 피트니스클럽 모델에서 탈피 커뮤니...,0
1,삼성전자 경기 기캠퍼스 주차타워에 설치된 태양광 발전 시설. 삼성전자 제공 전력 소...,0
2,한국경제신문의 프리미엄 스타트업 미디어 플랫폼 한경 긱스가 13일 스타트업 뉴스를 ...,0
3,꿈을 시각주는 기계. 사진바이두제 꿈이 정말 그림으로 그려지나요 지난달 24일 중국...,0
4,이미지11일 기자는 추석 연를 맞아 직접 텍스트를 이미지로 만들어주는 웹사이트 크레...,0


In [13]:

data_list = []
for q, label in zip(chatbot_data['newsfulltext'], chatbot_data['theme'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(len(data_list))

5535


In [14]:

#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])

4151
1384
[' -아이오닉 디 유니크, 충전·카 케어·레저·생활 및 문화 중 1개 선택 가능 현대자동차가 아이오닉5 구매자를 위한 멤버십 서비스인 \'아이오닉 디 유니크(IONIQ The Unique)\'를 출시한다고 26일 밝혔다. 아이오닉 디 유니크는 소비자 취향에 따라 충전, 카 케어, 레저, 생활·문화 등 4개 카테고리에 포함된 9개 서비스 중 한 가지를 선택해 무료로 이용할 수 있다. 충전 카테고리는 7㎾, 또는 11㎾ 홈 충전기 제공, 30만원 상당의 공용 충전 선불 카드 제공이 있다. 카 케어는 방문세차 이용권, 맞춤형 세차 이용권, PPF 시공 이용권 등의 서비스를 고를 수 있다. 레저는 글램핑 숙박 이용권, 차박 캠핑용품과 밀키트 제공 서비스가 있으며 생활·문화는 스트리밍 서비스 연간 이용권, 커피·와인·친환경 키트 등의 정기구독 서비스 지원이 있다. 서비스는 아이오닉5 출고자가 블루멤버스 회원 가입 후 마이현대 어플리케이션을 통해 신청할 수 있다. 현대차 관계자는 "앞으로도 아이오닉5 멤버십 이용자에게 차별화한 서비스를 지속적으로 제공할 것"이라고 말했다. 한편, 현대차는 충전 편의성 개선을 위해 소비자의 전기차를 픽업해 충전 후 전달하는 픽업 앤 충전 서비스를 4월부터 서울에서 시범 운영한다. 또한, 18분 만에 배터리를 10%에서 80%까지 충전할 수 있는 350㎾ 급속 충전기 120기를 올해 전국 고속도로 휴게소 12곳과 주요 도심 8곳에 설치할 계획이다.구기성 기자 kksstudio@autotimes.co.kr▶ 현대차 스타리아, 사전계약 첫날 1만대 넘겼다▶ 링컨 네비게이터 "플래그십 세단같은 SUV"▶ 현대차 7세대 그랜저, 2022년 대형차급으로 복귀▶ 기아 K8, 배기량 키운 LPG 그랜저 보다 싸다 ', '16']
[' 위챗, 웨이신 등 소셜미디어를 운영하는 중국 정보기술 기업 텐센트의 분기 매출이 상장 이 처음으로 감소다. 중국의 도시 봉쇄와 게임 규제 여파가 겹쳐 사업 대에 어려움을 겪었기 때문이다.17일지시간 텐센

In [15]:

#KoBERT 입력 데이터로 만들기
### 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
## Setting parameters 에포크 10
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
#토큰화와 패딩 
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

data_train[0]   # 첫 번째는 패딩된 시퀀스, 두 번째는 길이와 타입에 대한 내용, 세 번재는 어텐션 마스크 시퀀스

data_test[1]

(array([   2, 1639, 6896, 1564, 5760, 1316, 6216,  522,  597,  517,   40,
         517, 5760, 4760, 6500, 4249, 7788, 3862,  517,   54, 1437,  529,
        7028, 2986, 7794, 2103, 2852, 1109, 7757,  712,  517,  164, 6556,
         517,  498,  517, 6896, 4985, 7866, 1850,  517,   54, 1323,  714,
        4299, 1564, 5760, 4355, 5859, 1370, 6488, 7327, 3165,  517,   46,
        2368,  517, 6184, 2155, 6116, 2362, 6079, 5019,  517,  502,  993,
        1966,  517,   54,  714,  517,  164, 6556, 1109, 5564, 6903,  553,
        5713, 1109, 7088, 2220, 7083, 3647, 7342, 7095, 4871, 7096, 1698,
        1459, 6855, 7321, 3347, 5859, 1673, 1550,  915, 1267, 5330, 3862,
         517,  502,  993, 5019,  517,   54, 1107,  517,  478, 1103, 7191,
         517,  478, 4948, 6441, 3272,  687,  358, 5491, 5564,  164, 6556,
        7086,  517, 5472, 7229, 7637, 6399,    3], dtype=int32),
 array(128, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0

In [21]:
## torch 형식의 dataset 
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [22]:
### KoBERT 학습모델 
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=33,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [23]:
#BERT 모델 불러오기 (.to(device) 추가함, model을 어느 디바이스로)
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [24]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [25]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [26]:
#정확도 측정을 위한 함수 정의 (train_dataloader 같은 셀로 처리)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [27]:
# class EarlyStopping:
#     """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
#     def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
#         """
#         Args:
#             patience (int): validation loss가 개선된 후 기다리는 기간
#                             Default: 7
#             verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
#                             Default: False
#             delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
#                             Default: 0
#             path (str): checkpoint저장 경로
#                             Default: 'checkpoint.pt'
#         """
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_score = None
#         self.early_stop = False
#         self.val_loss_min = np.Inf
#         self.delta = delta
#         self.path = path

#     def __call__(self, val_loss, model):

#         score = -val_loss

#         if self.best_score is None:
#             self.best_score = score
#             self.save_checkpoint(val_loss, model)
#         elif score < self.best_score + self.delta:
#             self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
#             if self.counter >= self.patience:
#                 self.early_stop = True
#         else:
#             self.best_score = score
#             self.save_checkpoint(val_loss, model)
#             self.counter = 0

#     def save_checkpoint(self, val_loss, model):
#         '''validation loss가 감소하면 모델을 저장한다.'''
#         if self.verbose:
#             print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
#         torch.save(model.state_dict(), self.path)
#         self.val_loss_min = val_loss

In [28]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
        
    # early_stopping object의 초기화
    # early_stopping = EarlyStopping(patience = patience, verbose = True)

    # for epoch in range(1, n_epochs + 1):

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/65 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.5807788372039795 train acc 0.03125
epoch 1 train acc 0.11003933566433566


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/22 [00:00<?, ?it/s]

...

epoch 9 test acc 0.812784090909091


  0%|          | 0/65 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.4026421308517456 train acc 0.90625
epoch 10 train acc 0.9560096153846154


  0%|          | 0/22 [00:00<?, ?it/s]

epoch 10 test acc 0.812784090909091


In [30]:
import os
os.chdir('/content/drive/MyDrive/models/')
os.getcwd()

path = '/content/drive/MyDrive/models/'
torch.save(model, path + '33Cat_model.pt')  # 전체 모델 저장

torch.save(model.state_dict(), '33Cat_model_state_dict.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '33Cat_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [31]:
# 불러오기 편하라고 둔 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

#★★★현재경로가 model이 있는 폴더여야함★★★
import os
os.chdir('/content/drive/MyDrive/models/')

model1 = torch.load('33Cat_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('33Cat_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('33Cat_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/models/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [46]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            test_eval.append(np.argmax(logits))
    
    return test_eval[0]

end = 1
while end == 1 :
    sentence = input("카테고리를 알고싶나요? : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

카테고리를 알고싶나요? : 바이오니아의 주가가 급등세다. 탈모화장품 코스메르나가 유럽 화장품 인증인 CPNP(온라인 화장품 등록 포털) 등록을 사실상 확정 지었다는 소식의 영향으로 풀이된다. 15일 오전 9시 18분 바이오니아는 전 거래일 대비 6250원(22.32%) 오른 3만4250원에 거래되고 있다.  이날 바이오니아에 따르면 CPNP 등록 절차 중 가장 중요한 과정인 안전성 보고서(CPSR)를 확보했다. 유럽연합(EU) 내 화장품 원료 관리가 더욱 강화되면서 CPSR는 유럽 시장 진출의 성패를 가르는 요소로 떠올랐는데 독일 더마테스트사의 박사급 안전성 평가사들이 만장일치로 코스메르나의 안전성을 인정한다는 서명을 완료했으며 최근 안전성 보고서를 수령했다는 게 회사 측 설명이다.  안전성 평가사들은 원료 물질안전보건자료(MSDS)는 물론 국내와 유럽 인체적용 시험 데이터와 패치테스트 결과 등을 3개월 간 엄격하게 검토했다. 제품의 안전성과 효능을 입증한 논문이 네이처 산하 '사이언티픽 리포트'에 게재된 점도 평가에 긍정적으로 작용한 것으로 보인다.  바이오니아 관계자는 "유럽 CPNP 등록에 있어 가장 중요한 부분은 안전성 평가"라며 "글로벌 스탠다드인 독일 더마테스트 안전성 평가사들로부터 제품과 원료 모두 안전하다는 서명을 받아 사실상 CPNP 등록은 확정지은 셈" 이라고 말했다.  바이오니아는 코스메르나의 강점을 앞세워 탈모 관련 시장 점유율을 확대해 나갈 방침이다.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/1 [00:00<?, ?it/s]

25


카테고리를 알고싶나요? : 위더스제약 (KQ:330350)은 작년 7월 코스닥에 입성했습니다. 전문의약품 생산이 매출의 대부분을 차지합니다. 위더스제약은 새로운 먹거리로 탈모 치료제를 선택했습니다. 위더스제약이 탈모 치료제를 생산하는 설비를 구축하겠다고 23일 공시했습니다.   ◇위더스제약 [공장 신설 투자금액 195억원(자기자본 대비 25.4%), 투자 기간(21.06.23~22.07.21) =탈모는 성별, 나이를 가리지 않습니다. 국내 탈모 치료제 시장은 1500억원대입니다. 위더스제약은 기존의 경구용(經口用·먹는 치료제)이 아닌 주사용 탈모 치료제를 개발하겠다는 구상입니다. 이를 위해 대웅제약, 인벤티지랩과 위더스제약은 지난 3일 개발, 생산 협약을 체결했습니다. 향후 대웅제약은 임상 3상과 허가, 판매를 담당하고, 위더스제약은 생산을 담당하기로 했습니다.     =위더스제약은 생산에 필요한 선비를 구축하고자, 195억원을 투자하기로 했습니다. 투자 기간은 23일부터 내년 7월 21일까지입니다. =위더스제약이 생산하려는 주사용 탈모 치료제는 1회 주사로 약효가 최대 석달간 지속합니다. 기존의 탈모 치료약은 매일 복용하고, 양을 정확히 지켜야 하는 불편이 있었습니다. =위더스제약은 탈모 치료제의 임상 1상을 호주에서 7월 시작한다고 밝혔습니다. 위더스제약은 "호주를 선택한 이유는 앞으로 동남아 진출을 염두에 둔 행보"라고 밝혔습니다. =위더스제약이 코스닥에 상장하면서 약 160억원의 자금이 회사로 유입됐습니다. 상장 이후 위더스제약은 인벤티지랩에 20억원을 투자해, 지분 2.5%를 확보했습니다. 또한 기존 생산시설 증축에 100억원을 사용했습니다. =NH투자증권은 "위더스제약의 올해 매출로 582억원, 영업이익은 92억원을 각각 예상한다"고 밝혔습니다.


  0%|          | 0/1 [00:00<?, ?it/s]

25




KeyboardInterrupt: ignored